In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments/'

In [3]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

In [4]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

## (TEST) Checking actual delay (from the start of the 3 epochs)

In [6]:
df_cumdelay = pd.read_pickle(DF_PATH + 'cumu-delay/delay_d450_a0_b1_c0.pkl')
df_delay0 = df_cumdelay.loc[60, :]

df_instdelay = df_cumdelay.diff()
df_instdelay.loc[60, :] = df_delay0
df_instdelay

level_1,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)",...,"(3, 0, 2)","(3, 0, 3)","(3, 1, 0)","(3, 1, 1)","(3, 1, 2)","(3, 1, 3)","(3, 2, 0)","(3, 2, 1)","(3, 2, 2)","(3, 2, 3)"
t,,,,,,,,,,,,,,,,,,,,,
60,16.993678,16.105122,0.000000,8.848693,20.492820,60.662988,87.515538,35.037077,57.968495,45.670489,...,22.835512,20.492606,32.642628,28.189777,28.732533,18.564616,66.969246,55.546078,48.918024,32.700922
120,145.859194,84.420489,33.845692,57.796078,70.335512,151.804222,163.531076,37.728461,169.038363,126.011466,...,97.078654,23.784291,93.380197,53.618946,104.994213,30.086155,187.250536,115.156772,153.320296,58.743356
180,351.974711,166.090977,231.632769,57.796078,144.806794,196.738963,236.105229,57.765538,194.522610,135.471710,...,278.771753,43.110802,255.722460,82.476747,304.954136,57.570617,404.394664,182.425311,380.676811,107.761252
240,460.346550,328.261466,454.169846,74.894771,244.149486,258.309580,291.775075,57.765538,223.756858,148.181955,...,513.727282,60.634376,486.710994,180.494428,545.005572,80.579619,670.677025,357.607158,643.579083,146.860051
300,625.340227,436.076832,675.706922,193.789541,319.742177,339.336691,337.540613,77.302615,296.209601,182.997321,...,750.050257,70.599014,739.262216,337.056472,786.409130,99.449850,958.683991,599.857158,895.731354,210.413095
360,830.462066,606.642199,733.552615,344.684312,410.099228,364.950814,280.003536,50.302615,344.693849,173.892199,...,980.545542,71.651950,954.089906,534.954351,1007.351372,110.024003,1228.608873,802.508465,1125.634698,247.321582
420,929.833905,603.392199,782.052615,357.434312,478.727818,277.537827,215.679383,62.802615,273.225354,123.971710,...,1123.189456,108.745714,1196.112409,669.800471,1309.950493,146.872696,1468.351907,913.255465,1429.653022,388.251286
480,874.827583,663.037077,717.993999,361.283005,538.099228,352.673703,330.275075,103.206922,267.725354,160.181955,...,1254.606515,128.078761,1269.044685,886.896057,1339.901200,179.835619,1557.111766,1120.187312,1539.885877,585.444921
540,915.564938,597.576832,628.839691,382.033005,584.484998,554.016074,518.120767,202.839691,313.193849,180.642199,...,1233.085191,142.739713,1254.597835,1045.307544,1313.042542,203.709387,1547.057330,1365.745928,1719.081012,747.909773


In [35]:
list(df_instdelay.sum(axis=1)[[60, 240, 420]])

[586.9112730390056, 8391.820188598369, 19665.289327046718]

## (TEST) Checking expected delay (from the start of 3 epochs)

In [24]:
df_milp = pd.read_pickle(DF_PATH + 'milp-realtime-results.pkl')
df_milp.head(10)

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,174.118326,767.272152,51.278481,382.056221,450,True,0.80,0.10,0.10,Delay priority,0
5,456.304917,767.272152,51.278481,-598.035046,450,True,0.45,0.10,0.45,Delay-Flow priority,0
3,343.920306,767.272152,51.278481,49.987495,450,True,0.45,0.45,0.10,Delay-Throughput priority,0
6,917.078324,767.272152,51.278481,-487.789937,450,True,0.33,0.33,0.33,Equal priority,0
2,891.892373,801.107595,54.000000,-1585.763682,450,True,0.10,0.10,0.80,Flow priority,0
0,0.227479,743.113924,66.835443,66.835443,450,False,0.00,0.00,0.00,Parent model,0
1,415.136684,791.607595,55.000000,-291.456345,450,True,0.10,0.80,0.10,Throughput priority,0
4,596.610098,791.607595,55.000000,-937.523394,450,True,0.10,0.45,0.45,Throughput-Flow priority,0
1,5.594564,2080.658228,60.000000,1209.951345,900,True,0.80,0.10,0.10,Delay priority,0
6,6.865374,2080.658152,60.000003,-643.922642,900,True,0.45,0.10,0.45,Delay-Flow priority,0


In [37]:
df_milp[(df_milp.demand == 450) & (df_milp.model_type == 'Throughput priority')].delay

[791.6075949367089, 4326.65822785, 6555.15189873]

## Combining expected and actual results

In [7]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [8]:
delay_dfs = []
throughput_dfs = []

df_milp = pd.read_pickle(DF_PATH + 'milp-realtime-results.pkl')
df_delay = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])
df_thru = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])

for demand in _demands:
    for weights in _weights:

        # Get actual values
        
        df_cumdelay_tmp = pd.read_pickle(DF_PATH + 'cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instdelay_tmp = df_cumdelay_tmp.diff()
        df_instdelay_tmp.loc[60, :] = df_cumdelay_tmp.loc[60, :]
        actual_delay = list(df_instdelay_tmp.sum(axis=1)[[60, 240, 420]])
        
        df_cumthru_tmp = pd.read_pickle(DF_PATH + 'cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instthru_tmp = df_cumthru_tmp.diff()
        df_instthru_tmp.loc[60, :] = df_cumthru_tmp.loc[60, :]
        actual_thru = list(df_instthru_tmp.sum(axis=1)[[60, 240, 420]])

        # Get expected values
        
        expected_delay = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].delay)
        expected_thru = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].throughput)
        
        # Combine into a dataframe
        
        for i in range(3):
            df_delay.loc[-1] = [demand, _model_type[weights], expected_delay[i], actual_delay[i], i]
            df_thru.loc[-1] = [demand, _model_type[weights], expected_thru[i], actual_thru[i], i]
            df_delay.index += 1
            df_thru.index += 1

In [9]:
df_delay.to_pickle(DF_PATH + "delay_comparison.pkl")
df_thru.to_pickle(DF_PATH + "throughput_comparison.pkl")